In [1]:
from strategy import *
SAVE_IMG = True

def IPDRoundRobin(k_strategies, num_iter, changing_str=False, against_itself=False, plot=False):
    n = k_strategies.size

    # initialize players with given strategies
    players = np.array([MultiPlayer(k, changing_str) for k in k_strategies])

    # each player plays against another in a round robin scheme
    if(plot):
        plt.figure(figsize=(12,5))
    
    p = {obj:[0] * num_iter for obj in players}
    
    for (i, p1) in zip(np.arange(n), players):
        start = i if against_itself else i+1
        for (j, p2) in zip(np.arange(start, n), players[start:]):
            p1.clear_history()
            p2.clear_history()
            p1.play_iter(p2, num_iter)
            if(plot):
                p[p1] += np.cumsum(p1.payoffHist)
                p[p2] += np.cumsum(p2.payoffHist)
    if plot:
        for i in p:
            plt.plot(p[i],label=i.s)
            plt.xlabel('Iteration')
            plt.ylabel('Cum. reward')
        plt.title("Evolution of the game")
        plt.legend( bbox_to_anchor=(1, 1))
        if SAVE_IMG:
            plt.savefig('../img/ipdmp/ipdmp-evolution-of-game-{}.eps'.format(len(p)),format='eps')
            plt.close()
        else:
            plt.show()
    
    
    # for cipdmp: exit without computing points and matches df
    # dirty workaround: fix later when cipdmp points comp is well defined (now they are done in main)
    if changing_str:
        return players

    # calculate ranking and matches dataframes
    # has to be done after the tournament
    ranking_df = pd.DataFrame() # all points gained by players
    matches_df = pd.DataFrame() # all matches played sorted by time

    for (i, p) in zip(np.arange(n), players):
        points = p.get_points()
        df = pd.DataFrame(
            [[p.s, int(points[-1]), p, p.s.id]],
            columns=['Player','points', 'rrp', 'labels']
        )
        ranking_df = ranking_df.append(df)
        ranking_df = ranking_df.sort_values(['points'], ascending=[False])

        for j in range(i, len(p.results)):
            # can now access any property from p1 or p2 for plots
            # each match can be explored

            df = pd.DataFrame(
                    [[p.s, p.prevOpponent[j].s, p.results[j], p.prevOpponent[j].results[i]]],
                    columns=['p1','p2','p1-score','p2-score']
            )
            matches_df = matches_df.append(df)

    players = np.array(ranking_df['rrp'])
    ranking_df = ranking_df[['Player','points', 'labels']]
    return players, ranking_df, matches_df

def main():
    np.random.seed(100)
    pd.set_option('display.max_columns', None)

    NUM_ITER = 50
    NUM_PLAYERS = 20
    NUM_REPETITIONS = 10
    print("Testing round-robin tournament with {}-people".format(NUM_PLAYERS))

    # define k for strategy probabilities
    k_strategies = Strategy.generatePlayers(NUM_PLAYERS, replace=(NUM_PLAYERS != 8))

    repeated_players = []
    for i in range(NUM_REPETITIONS):
        players, ranking_df, matches_df = IPDRoundRobin(k_strategies, NUM_ITER, plot=(i==(NUM_REPETITIONS-1))) # no strategy change, not against itself
        repeated_players.append(players)

        # print(ranking_df.to_latex(index=False))
        # print(matches_df.to_latex(index=False))

    # save points and other stuff if necessary
    saved_points = []

    # save plots
    for players in repeated_players:
        for p in players:
            # save points for each repetition
            points = p.get_points()
            saved_points.append(int(points[-1]))

    # box plot of single match
    one_round_results = [p.results for p in players]
    one_round = pd.DataFrame(one_round_results).T
    meds = one_round.median().sort_values(ascending=False)
    one_round = one_round[meds.index]
    one_round.boxplot()
    plt.xticks(np.arange(NUM_PLAYERS)+1, [players[p].s for p in meds.index], rotation=90)
    plt.suptitle('Mean and variance for each type vs the other players \n One complete round')
    plt.ylabel('Points')
    plt.xlabel('Player')
    if SAVE_IMG:
        plt.savefig('../img/ipdmp/ipdmp-boxplot-single-match-{}.eps'.format(NUM_PLAYERS),format='eps')
        plt.close()
    else:
        plt.show()

    # box plot of all points
    saved_points = pd.DataFrame(np.reshape(saved_points, (NUM_REPETITIONS, int(len(saved_points)/NUM_REPETITIONS))))
    meds = saved_points.median().sort_values(ascending=False)
    saved_points = saved_points[meds.index]
    saved_points.boxplot()
    plt.xticks(np.arange(NUM_PLAYERS)+1, [players[p].s for p in meds.index], rotation=90)
    plt.suptitle(("Mean and variance for each type at the end of the tournament - {} repetitions").format(NUM_REPETITIONS))
    plt.ylabel('Points')
    plt.xlabel('Player')
    if SAVE_IMG:
        plt.savefig('../img/ipdmp/ipdmp-boxplot-final-points-{}.eps'.format(NUM_PLAYERS),format='eps')
        plt.close()
    else:
        plt.show()
    
if __name__ == "__main__":
    main()

Testing round-robin tournament with 20-people


In [ ]:
from ipdmp import IPDRoundRobin
from strategy import *
import time
    
def main():
    np.random.seed(100)
    pd.set_option('display.max_columns', None)

    SAVE_IMG = True

    NUM_ITER = 50
    NUM_PLAYERS = 50
    PERCENTAGE = 0.3
    print("Testing repeated round-robin tournament with {}-people".format(NUM_PLAYERS))

    k_strategies = Strategy.generatePlayers(NUM_PLAYERS, replace=True)

    NUM_REPETITIONS = 0
    MAX_ALLOWED = 5
    repeated_players = []
    strategies_df = pd.DataFrame() # strategies evolution

    # while not np.array_equal(k_strategies, np.repeat(k_strategies[0], k_strategies.size)):
    # this is the largest number of elements of a strategy
    while np.unique(k_strategies, return_counts=True)[1].max() < k_strategies.size*3/4 and NUM_REPETITIONS < MAX_ALLOWED:
        NUM_REPETITIONS += 1
        players, ranking_df, matches_df = IPDRoundRobin(k_strategies, NUM_ITER) # no strategy change, not against itself
        repeated_players.append(players)

        # create strategies history
        unique, counts = np.unique(k_strategies, return_counts=True)
        df = pd.DataFrame([counts],columns=unique)
        strategies_df = strategies_df.append(df)

        for i in range(0,len(players)):
            draw = np.random.uniform(0,1)
            #if(i < int(NUM_PLAYERS * PERCENTAGE)):
            #    if(draw > 0.2): #TODO we can also put something like i/len(players)
            #        k_strategies = np.append(k_strategies, players[i].s.id)
            #elif(i < 2*int(NUM_PLAYERS * PERCENTAGE)):
            #    if(draw > 0.5):
            #        k_strategies = np.append(k_strategies, players[i].s.id)
            #else:
            #    if(draw > 0.8):
            #        k_strategies = np.append(k_strategies, players[i].s.id)
            if(draw > i/len(players)):
                k_strategies = np.append(k_strategies, players[i].s.id)
                
    if(np.unique(k_strategies, return_counts=True)[1].max() > k_strategies.size*3/4 ):
        print("Convergence speed of round-robin tournament is {} with {}-people".format(NUM_REPETITIONS, NUM_PLAYERS))
    else:
        print("Convergence not reached")
        
    # save plots
    strategies_df = strategies_df.rename(index=str, columns={-3: "TitForTwoTat", -2: "GrimTrigger", -1: "TitForTat",
                                                                0: "Nice", 100: "Bad", 50: "Indifferent"})
    for c in strategies_df.columns:
        if str.isdigit(str(c)):
            if c > 50:
                strategies_df = strategies_df.rename(index=str, columns={c: "MainlyBad (k={})".format(c)})
            else:
                strategies_df = strategies_df.rename(index=str, columns={c: "MainlyNice (k={})".format(c)})

                
    strategies_df.index = np.arange(strategies_df.index.size)
    strategies_df = strategies_df.fillna(0)
    strategies_df.plot(figsize=(12,5))    
    plt.legend(ncol=int(len(strategies_df.columns)/10), bbox_to_anchor=(1, 1))
    plt.title('Strategies evolution')
    plt.ylabel('Number of strategies')
    plt.xlabel('Time')
    if SAVE_IMG:
        plt.savefig('../img/ripdmp-incr/ripdmp-evolution-increasing-pop-{}.eps'.format(NUM_PLAYERS),format='eps')
        plt.close()
    else:
        plt.show()

    for (r, players) in zip(np.arange(NUM_REPETITIONS), repeated_players):
        plt.figure(figsize=(12,5))
        for p in players:
            points = p.get_points()
            plt.plot(points, label=p.s)
            plt.title("Multi pl. game: {}".format(NUM_PLAYERS))
            plt.xlabel('Match number')
            plt.ylabel('Points')

        plt.legend(ncol=int(NUM_PLAYERS/10), bbox_to_anchor=(1, 1))

        if SAVE_IMG:
            plt.savefig('../img/ripdmp-incr/ripdmp-scores-increasing-pop-{}-r{}.eps'.format(NUM_PLAYERS, r),format='eps')
            plt.close()
        else:
            plt.show()
            
if __name__ == "__main__":
    main()

Testing repeated round-robin tournament with 50-people


In [ ]:
from ipdmp import IPDRoundRobin
from strategy import *

def main():
    np.random.seed(100)
    pd.set_option('display.max_columns', None)

    SAVE_IMG = True

    NUM_ITER = 100
    NUM_PLAYERS = 50
    PERCENTAGE = 0.3
    print("Testing repeated round-robin tournament with {}-people".format(NUM_PLAYERS))

    k_strategies = Strategy.generatePlayers(NUM_PLAYERS, replace=True)

    NUM_REPETITIONS = 0
    MAX_ALLOWED = 10
    repeated_players = []
    strategies_df = pd.DataFrame() # strategies evolution

    # while not np.array_equal(k_strategies, np.repeat(k_strategies[0], k_strategies.size)):
    # this is the largest number of elements of a strategy
    while np.unique(k_strategies, return_counts=True)[1].max() < k_strategies.size*3/4 and NUM_REPETITIONS < MAX_ALLOWED:
        NUM_REPETITIONS += 1
        players, ranking_df, matches_df = IPDRoundRobin(k_strategies, NUM_ITER) # no strategy change, not against itself
        repeated_players.append(players)

        # create strategies history
        unique, counts = np.unique(k_strategies, return_counts=True)
        df = pd.DataFrame([counts],columns=unique)
        strategies_df = strategies_df.append(df)

        # easy fix (depending on task)
        # add one winner strategy or multiple previous winners?
        for i in range(0,int(NUM_PLAYERS * PERCENTAGE)):
            k_strategies = np.append(k_strategies, players[i].s.id)
            k_strategies = np.delete(k_strategies,np.argmax(players[NUM_PLAYERS-i-1].s.id))
        #display(ranking_df)
        # display(matches_df)


    if(np.unique(k_strategies, return_counts=True)[1].max() > k_strategies.size*3/4 ):
        print("Convergence speed of round-robin tournament is {} with {}-people".format(NUM_REPETITIONS, NUM_PLAYERS))
    else:
        print("Convergence not reached")
        
    # save plots
    strategies_df = strategies_df.rename(index=str, columns={-3: "TitForTwoTat", -2: "GrimTrigger", -1: "TitForTat",
                                                                0: "Nice", 100: "Bad", 50: "Indifferent"})
    for c in strategies_df.columns:
        if str.isdigit(str(c)):
            if c > 50:
                strategies_df = strategies_df.rename(index=str, columns={c: "MainlyBad (k={})".format(c)})
            else:
                strategies_df = strategies_df.rename(index=str, columns={c: "MainlyNice (k={})".format(c)})

                
    strategies_df.index = np.arange(strategies_df.index.size)
    strategies_df = strategies_df.fillna(0)
    strategies_df.plot(figsize=(12,5))    
    plt.legend(ncol=int(len(strategies_df.columns)/10), bbox_to_anchor=(1, 1))
    plt.title('Strategies evolution')
    plt.ylabel('Number of strategies')
    plt.xlabel('Time')
    if SAVE_IMG:
        plt.savefig('../img/ripdmp-const/ripdmp-evolution-const-pop-{}.eps'.format(NUM_PLAYERS),format='eps')
        plt.close()
    else:
        plt.show()

    for (r, players) in zip(np.arange(NUM_REPETITIONS), repeated_players):
        plt.figure(figsize=(12,5))
        for p in players:
            points = p.get_points()
            plt.plot(points, label=p.s)
            plt.title("Multi pl. game: {}".format(NUM_PLAYERS))
            plt.xlabel('Match number')
            plt.ylabel('Points')

        plt.legend(ncol=int(NUM_PLAYERS/10), bbox_to_anchor=(1, 1))

        if SAVE_IMG:
            plt.savefig('../img/ripdmp-const/ripdmp-scores-const-pop-{}-r{}.eps'.format(NUM_PLAYERS, r),format='eps')
            plt.close()
        else:
            plt.show()
            
if __name__ == "__main__":
    main()